In [1]:
import pandas as pd
import numpy as np
#import talib as ta
import os

### Read Downloaded Kaggle Dataset:

In [2]:
fundamentals = pd.read_csv("./data/nyse/fundamentals.csv", parse_dates=['Period Ending'])
prices = pd.read_csv("./data/nyse/prices.csv", parse_dates=['date'])
prices_adj = pd.read_csv("./data/nyse/prices-split-adjusted.csv", parse_dates=['date'])
securities = pd.read_csv("./data/nyse/securities.csv", parse_dates=['Date first added'])

In [3]:
securities.rename(columns={'Ticker symbol':'symbol'},inplace=True)
securities.set_index(['symbol'], inplace=True, drop=True)
securities.sort_index(inplace=True)
securities.head()

,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
symbol,,,,,,,
A,Agilent Technologies Inc,reports,Health Care,Health Care Equipment,"Santa Clara, California",NaT,1090872
AAL,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",2015-03-23,6201
AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Roanoke, Virginia",2015-07-09,1158449
AAPL,Apple Inc.,reports,Information Technology,Computer Hardware,"Cupertino, California",1982-11-30,320193
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152


In [4]:
prices_adj.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [5]:
prices_adj.tail()

,date,symbol,open,close,low,high,volume
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0
851263,2016-12-30,FTV,54.200001,53.630001,53.389999,54.480000,705100.0


In [6]:
symbols = sorted(prices_adj['symbol'].unique())
prices_adj.set_index(['symbol', 'date'], inplace=True, drop=True)
prices_adj.sort_index(inplace=True)

In [7]:
prices_adj["percent_change_close-close"]=(prices_adj['close'].shift(1)-prices_adj['close'])/prices_adj['close']
prices_adj["percent_change_close-open"]=(prices_adj['open'].shift(1)-prices_adj['close'])/prices_adj['close']
prices_adj["percent_change_close-low"]=(prices_adj['low'].shift(1)-prices_adj['close'])/prices_adj['close']
prices_adj["percent_change_close-high"]=(prices_adj['high'].shift(1)-prices_adj['close'])/prices_adj['close']

print(prices_adj.head())

                        open      close        low       high     volume  \
symbol date                                                                
A      2010-01-04  22.453504  22.389128  22.267525  22.625180  3815500.0   
       2010-01-05  22.324749  22.145923  22.002861  22.331903  4186000.0   
       2010-01-06  22.067240  22.067240  22.002861  22.174536  3243700.0   
       2010-01-07  22.017168  22.038626  21.816881  22.045780  3095100.0   
       2010-01-08  21.917024  22.031474  21.745350  22.067240  3733900.0   

                   percent_change_close-close  percent_change_close-open  \
symbol date                                                                
A      2010-01-04                         NaN                        NaN   
       2010-01-05                    0.010982                   0.013889   
       2010-01-06                    0.003566                   0.011669   
       2010-01-07                    0.001298                   0.001298   
       2010

In [8]:
daily_prices_by_symbol = prices_adj.unstack('date')
daily_prices_by_symbol['percent_change_close-close',pd.Timestamp('2010-01-04')]=np.NaN
daily_prices_by_symbol['percent_change_close-open',pd.Timestamp('2010-01-04')]=np.NaN
daily_prices_by_symbol['percent_change_close-high',pd.Timestamp('2010-01-04')]=np.NaN
daily_prices_by_symbol['percent_change_close-low',pd.Timestamp('2010-01-04')]=np.NaN
daily_price_changes_by_symbol=daily_prices_by_symbol[['percent_change_close-close','percent_change_close-open','percent_change_close-high','percent_change_close-low']]
daily_price_changes_by_symbol

percent_change_close-close                                              \
date                   2010-01-04 2010-01-05 2010-01-06 2010-01-07 2010-01-08   
symbol                                                                          
A                             NaN   0.010982   0.003566   0.001298   0.000325   
AAL                           NaN  -0.101695   0.043222  -0.028626   0.019455   
AAP                           NaN   0.005979  -0.008644   0.000247  -0.003937   
AAPL                          NaN  -0.001726   0.016163   0.001852  -0.006604   
ABBV                          NaN        NaN        NaN        NaN        NaN   
ABC                           NaN   0.007186   0.009546   0.016298  -0.010749   
ABT                           NaN   0.008145  -0.005523  -0.008216  -0.005086   
ACN                           NaN  -0.006142  -0.010519   0.000936   0.003993   
ADBE                          NaN  -0.016180   0.002127   0.019789   0.005451   
ADI                           NaN   0.001581   0.001901   0.007985  -0.005716   
ADM                           NaN  -0.005373   0.002535   0.010567   0.012646   
ADP                           NaN   0.005399   0.002353   0.000471   0.001414   
ADS                           NaN  -0.001969  -0.004974  -0.009110  -0.010638   
ADSK                          NaN   0.015427  -0.002368  -0.005495  -0.029703   
AEE                           NaN   0.003978   0.006919   0.009559   0.007034   
AEP                           NaN   0.011581  -0.010031  -0.008525  -0.011794   
AES                           NaN   0.010347   0.010456  -0.000746  -0.027576   
AET                           NaN   0.014448   0.004012  -0.030811   0.022324   
AFL                           NaN  -0.028192  -0.008708  -0.010619   0.010119   
AGN                           NaN   0.010028  -0.003248   0.008060   0.007359   
AIG                           NaN   0.019093   0.006520   0.019594  -0.025903   
AIV                           NaN  -0.024450  -0.017417  -0.035342   0.023725   
AIZ                           NaN  -0.028976   0.002906  -0.012121  -0.000638   
AJG                           NaN   0.005403  -0.000450   0.006341  -0.002710   
AKAM                          NaN  -0.028850   0.008311  -0.000378   0.000000   
ALB                           NaN   0.001841  -0.001838   0.007139  -0.014848   
ALK                           NaN  -0.051930   0.009053   0.005240   0.020259   
ALL                           NaN  -0.016176  -0.000323  -0.005148  -0.004165   
ALLE                          NaN        NaN        NaN        NaN        NaN   
ALXN                          NaN   0.014508  -0.002517  -0.003761  -0.001877   
...                           ...        ...        ...        ...        ...   
VZ                            NaN  -0.001800   0.044486   0.005988  -0.000630   
WAT                           NaN   0.013818  -0.001806  -0.004251  -0.000817   
WBA                           NaN   0.008108   0.007625  -0.005956  -0.001352   
WDC                           NaN  -0.025829  -0.000860   0.019961  -0.005020   
WEC                           NaN   0.013244  -0.010085   0.002021  -0.005827   
WFC                           NaN  -0.026719  -0.001423  -0.035015   0.009355   
WFM                           NaN   0.008327  -0.011453   0.004313  -0.005007   
WHR                           NaN   0.002079   0.008882  -0.021015  -0.020814   
WLTW                          NaN        NaN        NaN        NaN        NaN   
WM                            NaN   0.004411   0.000294  -0.002347  -0.004673   
WMB                           NaN  -0.012340  -0.031430   0.007133  -0.007961   
WMT                           NaN   0.010058   0.002240  -0.000560   0.005063   
WRK                           NaN        NaN        NaN        NaN        NaN   
WU                            NaN   0.004208   0.000526  -0.031107  -0.009596   
WY                            NaN  -0.020787   0.011181   0.004492   0.009295   
WYN                           NaN  -0.

In [9]:
print(daily_price_changes_by_symbol.shape)
num_days_missing=daily_price_changes_by_symbol['percent_change_close-close'].isnull().sum(1)
print(sum(num_days_missing>1))
print(num_days_missing[num_days_missing>1])
daily_price_changes_by_symbol=daily_price_changes_by_symbol.dropna(1,'all')
print(daily_price_changes_by_symbol)
daily_price_changes_by_symbol=daily_price_changes_by_symbol.dropna()
print(daily_price_changes_by_symbol)
print(daily_price_changes_by_symbol.shape)

(501, 7048)
33
symbol
ABBV     754
ALLE     976
CFG     1258
COTY     866
CSRA    1478
DLPH     754
EVHC     909
FB       754
FBHS     754
FTV     1636
GM       222
HCA      754
HPE     1458
KHC     1384
KMI      754
KORS     754
LYB       79
MNK      868
MPC      754
NAVI    1258
NLSN     754
NWS      870
NWSA     870
PSX      754
PYPL    1384
QRVO    1258
SYF     1258
TDG      754
TRIP     754
WLTW    1511
WRK     1377
XYL      754
ZTS      775
dtype: int64
       percent_change_close-close                                              \
date                   2010-01-05 2010-01-06 2010-01-07 2010-01-08 2010-01-11   
symbol                                                                          
A                        0.010982   0.003566   0.001298   0.000325  -0.000649   
AAL                     -0.101695   0.043222  -0.028626   0.019455   0.019841   
AAP                      0.005979  -0.008644   0.000247  -0.003937   0.009940   
AAPL                    -0.001726   0.016163   0.0

       percent_change_close-close                                              \
date                   2010-01-05 2010-01-06 2010-01-07 2010-01-08 2010-01-11   
symbol                                                                          
A                        0.010982   0.003566   0.001298   0.000325  -0.000649   
AAL                     -0.101695   0.043222  -0.028626   0.019455   0.019841   
AAP                      0.005979  -0.008644   0.000247  -0.003937   0.009940   
AAPL                    -0.001726   0.016163   0.001852  -0.006604   0.008900   
ABC                      0.007186   0.009546   0.016298  -0.010749  -0.011010   
ABT                      0.008145  -0.005523  -0.008216  -0.005086  -0.005061   
ACN                     -0.006142  -0.010519   0.000936   0.003993   0.000941   
ADBE                    -0.016180   0.002127   0.019789   0.005451   0.013256   
ADI                      0.001581   0.001901   0.007985  -0.005716   0.005749   
ADM                     -0.0

In [10]:
filtered_securities=securities[list(map(lambda x: x in daily_price_changes_by_symbol.index.values,securities.index.values))]
print(filtered_securities['GICS Sector'])
sector_labels=np.array(filtered_securities['GICS Sector'])
sub_sector_labels=np.array(filtered_securities['GICS Sub Industry'])
print(np.shape(sector_labels))
print(np.shape(sub_sector_labels))

symbol
A                       Health Care
AAL                     Industrials
AAP          Consumer Discretionary
AAPL         Information Technology
ABC                     Health Care
ABT                     Health Care
ACN          Information Technology
ADBE         Information Technology
ADI          Information Technology
ADM                Consumer Staples
ADP          Information Technology
ADS          Information Technology
ADSK         Information Technology
AEE                       Utilities
AEP                       Utilities
AES                       Utilities
AET                     Health Care
AFL                      Financials
AGN                     Health Care
AIG                      Financials
AIV                     Real Estate
AIZ                      Financials
AJG                      Financials
AKAM         Information Technology
ALB                       Materials
ALK                     Industrials
ALL                      Financials
ALXN                 

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier

from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.cluster import KMeans
from sklearn.mixture import GMM
from sklearn.cluster import SpectralClustering

In [12]:
daily_close=np.array(daily_price_changes_by_symbol['percent_change_close-close'])
daily_all=np.array(daily_price_changes_by_symbol)

shuffle = np.random.permutation(np.arange(sector_labels.shape[0]))
daily_close, daily_all, sector_labels, sub_sector_labels = daily_close[shuffle], daily_all[shuffle], sector_labels[shuffle], sub_sector_labels[shuffle]

train_indices=list(range(374))
dev_indices=list(range(374,421))
test_indices=list(range(421,468))

In [13]:
print(daily_close.shape, daily_all.shape, sector_labels.shape, sub_sector_labels.shape)
print(daily_close[train_indices].shape)
print(daily_close[train_indices])
for k in map(lambda x: 2*x+1,range(5)):
    print('Results using all daily features with k =',k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(daily_all[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(daily_all[dev_indices]),sector_labels[dev_indices])
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(daily_all[dev_indices])==sector_labels[dev_indices])/47)

(468, 1761) (468, 7044) (468,) (468,)
(374, 1761)
[[-0.00089174  0.00899817 -0.02171367 ...,  0.00690099  0.0006445
   0.00994979]
 [-0.02572285 -0.00318026  0.0070056  ...,  0.00930999  0.00800442
   0.00834957]
 [ 0.01317258 -0.01060303  0.00354686 ...,  0.00930614 -0.01350074
   0.00529316]
 ..., 
 [-0.02332155  0.0085531  -0.00988003 ...,  0.01729664  0.00224716
  -0.01079708]
 [-0.01413296 -0.01504891 -0.03869801 ...,  0.01080504  0.01027397
  -0.00638024]
 [-0.03727715  0.00652529 -0.04068858 ...,  0.01533412  0.00828266
  -0.00875747]]
k = 1
                             precision    recall  f1-score   support

     Consumer Discretionary       0.33      1.00      0.50         1
           Consumer Staples       0.80      1.00      0.89         4
                     Energy       1.00      1.00      1.00         6
                 Financials       1.00      0.89      0.94         9
                Health Care       1.00      1.00      1.00         5
                Industrials   

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                             precision    recall  f1-score   support

     Consumer Discretionary       0.00      0.00      0.00         0
           Consumer Staples       0.80      0.67      0.73         6
                     Energy       0.67      1.00      0.80         4
                 Financials       1.00      0.80      0.89        10
                Health Care       1.00      1.00      1.00         5
                Industrials       1.00      0.36      0.53        11
     Information Technology       0.67      1.00      0.80         4
                  Materials       0.50      1.00      0.67         1
                Real Estate       0.67      1.00      0.80         2
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       1.00      1.00      1.00         4

                avg / total       0.89      0.77      0.78        47

k = 9
                             precision    recall  f1-score   support

     Consumer Discretion

In [18]:
for k in map(lambda x: 2*x+1,range(5)):
    print('Results using only daily close features with k =',k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(daily_close[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(daily_close[dev_indices]),sector_labels[dev_indices])
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(daily_close[dev_indices])==sector_labels[dev_indices])/47)

Results using only daily close features with k = 1
                             precision    recall  f1-score   support

     Consumer Discretionary       0.33      1.00      0.50         1
           Consumer Staples       0.80      1.00      0.89         4
                     Energy       1.00      1.00      1.00         6
                 Financials       1.00      0.89      0.94         9
                Health Care       1.00      1.00      1.00         5
                Industrials       0.50      0.50      0.50         4
     Information Technology       0.83      0.62      0.71         8
                  Materials       1.00      1.00      1.00         2
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       1.00      1.00      1.00         1
                  Utilities       1.00      1.00      1.00         4

                avg / total       0.90      0.87      0.88        47

Classification Accuracy: 0.872340425532
Results u

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                             precision    recall  f1-score   support

     Consumer Discretionary       0.33      1.00      0.50         1
           Consumer Staples       0.80      0.80      0.80         5
                     Energy       1.00      1.00      1.00         6
                 Financials       1.00      0.67      0.80        12
                Health Care       1.00      1.00      1.00         5
                Industrials       0.50      0.40      0.44         5
     Information Technology       0.67      0.80      0.73         5
                  Materials       0.50      1.00      0.67         1
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       1.00      1.00      1.00         4

                avg / total       0.87      0.81      0.82        47

Classification Accuracy: 0.808510638298
Results using only daily close features with k = 7
         

In [14]:
normalized_daily_close=(daily_close-daily_close.mean(0))/daily_close.std(0)
print(normalized_daily_close.shape)
normalized_daily_all=(daily_all-daily_all.mean(0))/daily_all.std(0)
print(normalized_daily_all.shape)

(468, 1761)
(468, 7044)


In [15]:
for k in map(lambda x: 2*x+1,range(5)):
    print('Results using all normalized daily features with k =',k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(normalized_daily_all[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(normalized_daily_all[dev_indices]),sector_labels[dev_indices])
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(normalized_daily_all[dev_indices])==sector_labels[dev_indices])/47)

k = 1
                             precision    recall  f1-score   support

     Consumer Discretionary       0.33      1.00      0.50         1
           Consumer Staples       0.80      1.00      0.89         4
                     Energy       1.00      1.00      1.00         6
                 Financials       1.00      1.00      1.00         8
                Health Care       0.80      1.00      0.89         4
                Industrials       0.50      0.29      0.36         7
     Information Technology       0.50      0.43      0.46         7
                  Materials       1.00      1.00      1.00         2
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       1.00      1.00      1.00         1
                  Utilities       1.00      1.00      1.00         4

                avg / total       0.80      0.81      0.80        47

Classification Accuracy: 0.808510638298
k = 3
                             precision    recall

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                             precision    recall  f1-score   support

     Consumer Discretionary       0.00      0.00      0.00         0
           Consumer Staples       0.80      0.67      0.73         6
                     Energy       1.00      1.00      1.00         6
                 Financials       1.00      0.80      0.89        10
                Health Care       1.00      1.00      1.00         5
                Industrials       0.75      0.38      0.50         8
     Information Technology       0.67      0.80      0.73         5
                  Materials       0.50      1.00      0.67         1
                Real Estate       0.67      1.00      0.80         2
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       1.00      1.00      1.00         4

                avg / total       0.87      0.79      0.81        47

Classification Accuracy: 0.787234042553
k = 7
                             precision    recall  f1-s

In [19]:
for k in map(lambda x: 2*x+1,range(5)):
    print('Results using only normalized daily close features with k =',k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(normalized_daily_close[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(normalized_daily_close[dev_indices]),sector_labels[dev_indices])
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(normalized_daily_close[dev_indices])==sector_labels[dev_indices])/47)

Results using only normalized daily close features with k = 1
                             precision    recall  f1-score   support

     Consumer Discretionary       0.33      1.00      0.50         1
           Consumer Staples       0.80      1.00      0.89         4
                     Energy       1.00      1.00      1.00         6
                 Financials       1.00      0.89      0.94         9
                Health Care       0.80      1.00      0.89         4
                Industrials       0.50      0.67      0.57         3
     Information Technology       1.00      0.60      0.75        10
                  Materials       1.00      1.00      1.00         2
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       1.00      1.00      1.00         1
                  Utilities       1.00      1.00      1.00         4

                avg / total       0.92      0.87      0.88        47

Classification Accuracy: 0.87234042553

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                             precision    recall  f1-score   support

     Consumer Discretionary       0.00      0.00      0.00         0
           Consumer Staples       0.80      0.67      0.73         6
                     Energy       0.67      1.00      0.80         4
                 Financials       1.00      0.62      0.76        13
                Health Care       1.00      1.00      1.00         5
                Industrials       0.50      0.29      0.36         7
     Information Technology       0.67      0.80      0.73         5
                  Materials       0.50      1.00      0.67         1
                Real Estate       0.67      1.00      0.80         2
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       1.00      1.00      1.00         4

                avg / total       0.81      0.72      0.74        47

Classification Accuracy: 0.723404255319
Results using only normalized daily close features with k = 

In [22]:
for alpha in map(lambda x: 10**(x-3),range(6)):
    model=BernoulliNB(alpha=alpha)
    model.fit(daily_all[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(daily_all[dev_indices]),sector_labels[dev_indices])
    print("Results using all daily features with alpha =",alpha)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(daily_all[dev_indices])==sector_labels[dev_indices])/47)

    model.fit(normalized_daily_all[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(normalized_daily_all[dev_indices]),sector_labels[dev_indices])
    print("Results using all normalized daily features with alpha =",alpha)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(normalized_daily_all[dev_indices])==sector_labels[dev_indices])/47)
    
    model=BernoulliNB(alpha=alpha)
    model.fit(daily_close[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(daily_close[dev_indices]),sector_labels[dev_indices])
    print("Results using only daily close features with alpha =",alpha)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(daily_close[dev_indices])==sector_labels[dev_indices])/47)

    model.fit(normalized_daily_close[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(normalized_daily_close[dev_indices]),sector_labels[dev_indices])
    print("Results using all daily features with alpha =",alpha)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(normalized_daily_close[dev_indices])==sector_labels[dev_indices])/47)

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Results using all daily features with alpha = 0.001
                             precision    recall  f1-score   support

     Consumer Discretionary       0.67      0.29      0.40         7
           Consumer Staples       0.80      1.00      0.89         4
                     Energy       0.83      1.00      0.91         5
                 Financials       1.00      1.00      1.00         8
                Health Care       1.00      1.00      1.00         5
                Industrials       0.75      0.75      0.75         4
     Information Technology       1.00      0.86      0.92         7
                  Materials       1.00      1.00      1.00         2
                Real Estate       0.67      1.00      0.80         2
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       0.75      1.00      0.86         3

                avg / total       0.86      0.85      0.84        47

Classification Accuracy: 0.851063829787
Results 

Results using all normalized daily features with alpha = 0.1
                             precision    recall  f1-score   support

     Consumer Discretionary       0.67      0.67      0.67         3
           Consumer Staples       0.80      0.80      0.80         5
                     Energy       0.83      1.00      0.91         5
                 Financials       1.00      1.00      1.00         8
                Health Care       1.00      1.00      1.00         5
                Industrials       0.75      0.50      0.60         6
     Information Technology       0.83      0.71      0.77         7
                  Materials       1.00      1.00      1.00         2
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       0.75      1.00      0.86         3

                avg / total       0.87      0.85      0.85        47

Classification Accuracy: 0.851063829787

Results using all daily features with alpha = 10
                             precision    recall  f1-score   support

     Consumer Discretionary       0.67      0.67      0.67         3
           Consumer Staples       0.80      0.67      0.73         6
                     Energy       1.00      1.00      1.00         6
                 Financials       1.00      1.00      1.00         8
                Health Care       1.00      1.00      1.00         5
                Industrials       0.50      0.50      0.50         4
     Information Technology       0.83      1.00      0.91         5
                  Materials       1.00      1.00      1.00         2
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       1.00      0.80      0.89         5

                avg / total       0.89      0.87      0.88        47

Classification Accuracy: 0.872340425532
Results usi

In [26]:
for n_estimators in range(18,35):
    model=RandomForestClassifier(n_estimators=n_estimators)
    model.fit(daily_all[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(daily_all[dev_indices]),sector_labels[dev_indices])
    print("Results using all daily features with n_estimators =",n_estimators)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(daily_all[dev_indices])==sector_labels[dev_indices])/47)

    model.fit(normalized_daily_all[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(normalized_daily_all[dev_indices]),sector_labels[dev_indices])
    print("Results using all normalized daily features with n_estimators =",n_estimators)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(normalized_daily_all[dev_indices])==sector_labels[dev_indices])/47)
    
    model=RandomForestClassifier(n_estimators=n_estimators)
    model.fit(daily_close[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(daily_close[dev_indices]),sector_labels[dev_indices])
    print("Results using only daily close features with n_estimators =",n_estimators)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(daily_close[dev_indices])==sector_labels[dev_indices])/47)

    model.fit(normalized_daily_close[train_indices],sector_labels[train_indices])
    result=classification_report(model.predict(normalized_daily_close[dev_indices]),sector_labels[dev_indices])
    print("Results using all daily features with n_estimators =",n_estimators)
    print(result)
    print("Classification Accuracy:",np.sum(model.predict(normalized_daily_close[dev_indices])==sector_labels[dev_indices])/47)

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Results using all daily features with n_estimators = 18
                             precision    recall  f1-score   support

     Consumer Discretionary       0.67      0.40      0.50         5
           Consumer Staples       0.80      0.80      0.80         5
                     Energy       0.67      1.00      0.80         4
                 Financials       1.00      1.00      1.00         8
                Health Care       1.00      0.83      0.91         6
                Industrials       0.50      0.40      0.44         5
     Information Technology       0.83      0.71      0.77         7
                  Materials       0.50      1.00      0.67         1
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       0.75      1.00      0.86         3

                avg / total       0.81      0.79      0.79        47

Classification Accuracy: 0.787234042553


C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Results using all normalized daily features with n_estimators = 18
                             precision    recall  f1-score   support

     Consumer Discretionary       0.67      0.22      0.33         9
           Consumer Staples       0.80      0.80      0.80         5
                     Energy       1.00      0.86      0.92         7
                 Financials       1.00      1.00      1.00         8
                Health Care       0.60      1.00      0.75         3
                Industrials       0.25      0.33      0.29         3
     Information Technology       0.67      1.00      0.80         4
                  Materials       0.50      0.50      0.50         2
                Real Estate       0.67      0.67      0.67         3
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       0.50      0.67      0.57         3

                avg / total       0.74      0.70      0.69        47

Classification Accuracy: 0.702127

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


...

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Results using all normalized daily features with n_estimators = 34
                             precision    recall  f1-score   support

     Consumer Discretionary       0.67      0.50      0.57         4
           Consumer Staples       0.80      1.00      0.89         4
                     Energy       0.83      1.00      0.91         5
                 Financials       1.00      0.89      0.94         9
                Health Care       1.00      1.00      1.00         5
                Industrials       0.75      0.43      0.55         7
     Information Technology       0.83      0.83      0.83         6
                  Materials       0.50      1.00      0.67         1
                Real Estate       1.00      1.00      1.00         3
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       0.75      1.00      0.86         3

                avg / total       0.85      0.83      0.83        47

Classification Accuracy: 0.829787

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Results using only daily close features with n_estimators = 34
                             precision    recall  f1-score   support

     Consumer Discretionary       0.67      0.40      0.50         5
           Consumer Staples       0.80      0.57      0.67         7
                     Energy       0.83      1.00      0.91         5
                 Financials       1.00      1.00      1.00         8
                Health Care       1.00      1.00      1.00         5
                Industrials       1.00      0.57      0.73         7
     Information Technology       0.83      1.00      0.91         5
                  Materials       0.00      0.00      0.00         0
                Real Estate       0.67      1.00      0.80         2
Telecommunications Services       0.00      0.00      0.00         0
                  Utilities       0.75      1.00      0.86         3

                avg / total       0.87      0.81      0.82        47

Classification Accuracy: 0.8085106382

C:\Users\adpra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
